# Analyze continuous assay prediction

In [1]:
import polars as pl

dat = pl.read_parquet("/dgx1nas1/storage/data/jess/repos/2024_09_09_Axiom_OASIS/1_snakemake/outputs/dino/mad_featselect/profiles/mad_featselect.parquet")

